In [105]:
import sklearn.utils
import pandas as pd
import numpy as np
from nn import NeuralNet
from sklearn.metrics import mean_squared_error

In [106]:
# standardization class
class Standardizer: 
    """ class version of standardization """
    def __init__(self, X, explore=False):
        self._mu = np.mean(X,0)  
        self._sigma = np.std(X,0)
        if explore:
            print ("mean: ", self._mu)
            print ("sigma: ", self._sigma)
            print ("min: ", np.min(X,0))
            print ("max: ", np.max(X,0))

    def set_sigma(self, s):
        self._sigma[:] = s

    def standardize(self,X):
        return (X - self._mu) / self._sigma 

    def unstandardize(self,X):
        return (X * self._sigma) + self._mu 
    

In [146]:
""" Neural Network 
        referenced NN code by Chuck Anderson in R and C++ 

                                by Jake Lee (lemin)

    example usage:
        X = numpy.array([0,0,1,0,0,1,1,1]).reshape(4,2)
        T = numpy.array([0,1,1,0,1,0,0,1]).reshape(4,2)

        nn = nnet.NeuralNet([2,3,2])
        nn.train(X,T, wprecision=1e-20, fprecision=1e-2)
        Y = nn.use(X)

"""
import numpy as np
import matplotlib.pyplot as plt
from grad import scg, steepest
from copy import copy
from util import Standardizer
from nn import NeuralNet

class NeuralNetLogReg(NeuralNet):
    """ neural network class for regression
        
        Parameters
        ----------
        nunits: list
            the number of inputs, hidden units, and outputs

        Methods
        -------
        set_hunit    
            update/initiate weights

        pack 
            pack multiple weights of each layer into one vector

        forward
            forward processing of neural network

        backward
            back-propagation of neural network

        train
            train the neural network

        use
            appply the trained network for prediction

        Attributes
        ----------
        _nLayers
            the number of hidden unit layers 

        rho
            learning rate

        _W
            weights
        _weights
            weights in one dimension (_W is referencing _weight)

        stdX
            standardization class for data
        stdT
            standardization class for target

        Notes
        -----
        
    """

    def __init__(self, nunits):

        self._nLayers=len(nunits)-1
        self.rho = [1] * self._nLayers
        self._W = []
        wdims = []
        lenweights = 0
        for i in range(self._nLayers):
            nwr = nunits[i] + 1
            nwc = nunits[i+1]
            wdims.append((nwr, nwc))
            lenweights = lenweights + nwr * nwc

        self._weights = np.random.uniform(-0.1,0.1, lenweights) 
        start = 0  # fixed index error 20110107
        for i in range(self._nLayers):
            end = start + wdims[i][0] * wdims[i][1] 
            self._W.append(self._weights[start:end])
            self._W[i].resize(wdims[i])
            start = end

        self.stdX = None
        self.stdT = None
        self.stdTarget = True

    def add_ones(self, w):
        return np.hstack((np.ones((w.shape[0], 1)), w))

    def get_nlayers(self):
        return self._nLayers

    def set_hunit(self, w):
        for i in range(self._nLayers-1):
            if w[i].shape != self._W[i].shape:
                print("set_hunit: shapes do not match!")
                break
            else:
                self._W[i][:] = w[i][:]

    def pack(self, w):
        return np.hstack(map(np.ravel, w))

    def unpack(self, weights):
        self._weights[:] = weights[:]  # unpack

    def cp_weight(self):
        return copy(self._weights)

    def RBF(self, X, m=None,s=None):
        if m is None: m = np.mean(X)
        if s is None: s = 2 #np.std(X)
        r = 1. / (np.sqrt(2*np.pi)* s)  
        return r * np.exp(-(X - m) ** 2 / (2 * s ** 2))

    def forward(self,X):
        t = X 
        Z = []

        for i in range(self._nLayers):
            Z.append(t) 
            if i == self._nLayers - 1:
                t = np.dot(self.add_ones(t), self._W[i])
            else:
                t = np.tanh(np.dot(self.add_ones(t), self._W[i]))
                #t = self.RBF(np.dot(np.hstack((np.ones((t.shape[0],1)),t)),self._W[i]))
                
        print("In Forward t:", t)
        return (t, Z)
        
    def backward(self, error, Z, T, lmb=0):
        delta = error
        N = T.size
        dws = []
        for i in range(self._nLayers - 1, -1, -1):
            rh = float(self.rho[i]) / N
            if i==0:
                lmbterm = 0
            else:
                lmbterm = lmb * np.vstack((np.zeros((1, self._W[i].shape[1])),
                            self._W[i][1:,]))
                
            print(Z[i].T.shape)
            dws.insert(0,(-rh * np.dot(self.add_ones(Z[i]).T, delta) + lmbterm))
            if i != 0:
                delta = np.dot(delta, self._W[i][1:, :].T) * (1 - Z[i]**2)
        return self.pack(dws)
    
    def softmax(self, z):
        if not isinstance(z, np.ndarray):
            z = np.asarray(z)
        f = np.exp(z) 
        return f / (np.sum(f, axis=1, keepdims=True) if len(z.shape) == 2 else np.sum(f))

    # for linear fx
    def g(self,Y):
#         print("In g X", X)
        return self.softmax(Y)

    def _errorf(self, T, Y):
        return T - Y
        
    def _objectf(self, T, Y, wpenalty):
        return 0.5 * np.mean(np.square(T - Y)) + wpenalty

    def train(self, X, T, **params):

        verbose = params.pop('verbose', False)
        # training parameters
        _lambda = params.pop('Lambda', 0.)

        #parameters for scg
        niter = params.pop('niter', 1000)
        wprecision = params.pop('wprecision', 1e-10)
        fprecision = params.pop('fprecision', 1e-10)
        wtracep = params.pop('wtracep', False)
        ftracep = params.pop('ftracep', False)

        # optimization
        optim = params.pop('optim', 'scg')

        if self.stdX == None:
            explore = params.pop('explore', False)
            self.stdX = Standardizer(X, explore)
        Xs = self.stdX.standardize(X)
        if self.stdT == None and self.stdTarget:
            self.stdT = Standardizer(T)
            T = self.stdT.standardize(T)
        
        def gradientf(weights):
            self.unpack(weights)
            Y,Z = self.forward(Xs)
            g_Y = self.g(Y)
#             Yl = np.argmax(g_Y, 1)
#             Tl = np.argmax(T, 1)
            error = self._errorf(T, g_Y)
            return self.backward(error, Z, T, _lambda)
            
        def optimtargetf(weights):
            """ optimization target function : MSE 
            """
            self.unpack(weights)
            #self._weights[:] = weights[:]  # unpack
            Y,_ = self.forward(Xs)
            Wnb=np.array([])
            for i in range(self._nLayers):
                if len(Wnb)==0: Wnb=self._W[i][1:,].reshape(self._W[i].size-self._W[i][0,].size,1)
                else: Wnb = np.vstack((Wnb,self._W[i][1:,].reshape(self._W[i].size-self._W[i][0,].size,1)))
            wpenalty = _lambda * np.dot(Wnb.flat ,Wnb.flat)
            return self._objectf(T, Y, wpenalty)

        if optim == 'scg':
            result = scg(self.cp_weight(), gradientf, optimtargetf,
                                        wPrecision=wprecision, fPrecision=fprecision, 
                                        nIterations=niter,
                                        wtracep=wtracep, ftracep=ftracep,
                                        verbose=False)
            self.unpack(result['w'][:])
            self.f = result['f']
        elif optim == 'steepest':
            result = steepest(self.cp_weight(), gradientf, optimtargetf,
                                nIterations=niter,
                                xPrecision=wprecision, fPrecision=fprecision,
                                xtracep=wtracep, ftracep=ftracep )
            self.unpack(result['w'][:])
        if ftracep:
            self.ftrace = result['ftrace']
        if 'reason' in result.keys() and verbose:
            print(result['reason'])

        return result

    def use(self, X, retZ=False):
        if self.stdX:
            Xs = self.stdX.standardize(X)
        else:
            Xs = X
        Y, Z = self.forward(Xs)
        if self.stdT is not None:
            Y = self.stdT.unstandardize(Y)
        if retZ:
            return Y, Z
        return Y





In [138]:
def partition(X, T, ratio=[0.8, 0.2]):
        assert(np.sum(ratio) == 1)
         # shuffle the data indices 
        idxs = np.random.permutation(X.index)
        # the number of samples 
        N = X.shape[0]
#         print(type(idxs))
        Xs = []
        Ts = []
        i = 0  # first index to zero
        for k, r in enumerate(ratio):
#             print("Pass 1")
            nrows = int(round(N * r))  # number of rows
            
            if k == len(ratio) -1:
                Xs.append(X.iloc[i:, :])
                Ts.append(T.iloc[i:,])
            else:
                row = i+nrows
                Xs.append(X.iloc[:row, :])
                Ts.append(T.iloc[:row,])
        
            i += nrows
    
        return Xs, Ts




In [128]:
df = pd.read_csv("weatherAUS.csv")
df = df.dropna(axis = 0, how ='any')

In [110]:
df = df.drop(['Date', 'Location','MaxTemp','WindGustDir','WindDir9am','WindDir3pm','WindSpeed9am','WindSpeed3pm','Temp9am','Temp3pm'], axis = 1)
df

,MinTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,RainToday,RISK_MM,RainTomorrow
5939,17.9,0.0,12.0,12.3,48.0,20.0,13.0,1006.3,1004.4,2.0,5.0,No,0.0,No
5940,18.4,0.0,14.8,13.0,37.0,30.0,8.0,1012.9,1012.1,1.0,1.0,No,0.0,No
5942,19.4,0.0,10.8,10.6,46.0,42.0,22.0,1012.3,1009.2,1.0,6.0,No,0.0,No
5943,21.9,0.0,11.4,12.2,31.0,37.0,22.0,1012.7,1009.1,1.0,5.0,No,0.0,No
5944,24.2,0.0,11.2,8.4,35.0,19.0,15.0,1010.7,1007.4,1.0,6.0,No,0.0,No
5945,27.1,0.0,13.0,0.0,43.0,26.0,19.0,1007.7,1007.4,8.0,8.0,No,0.0,No
5946,23.3,0.0,9.8,12.6,41.0,33.0,15.0,1011.3,1009.9,3.0,1.0,No,0.0,No
5947,16.1,0.0,14.6,13.2,37.0,25.0,9.0,1013.3,1009.2,1.0,1.0,No,0.0,No
5948,19.0,0.0,12.0,12.3,48.0,46.0,28.0,1008.3,1004.0,1.0,5.0,No,0.0,No
5949,19.7,0.0,11.0,12.7,41.0,61.0,14.0,1007.9,1005.8,1.0,5.0,No,0.0,No


In [111]:
# 'Location','MaxTemp','WindGustDir','WindDir9am','WindDir3pm','WindSpeed9am','WindSpeed3pm','Temp9am','Temp3pm'])

In [115]:
raindic = {'No' : 0, 'Yes' : 1}

df['RainToday'] = df['RainToday'].apply(lambda x: raindic[x])
df['RainTomorrow'] = df['RainTomorrow'].apply(lambda x: raindic[x])

In [116]:
df

,MinTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,RainToday,RISK_MM,RainTomorrow
136197,17.9,0.0,12.0,12.3,48.0,20.0,13.0,1006.3,1004.4,2.0,5.0,0,0.0,0
118358,18.4,0.0,14.8,13.0,37.0,30.0,8.0,1012.9,1012.1,1.0,1.0,0,0.0,0
76886,19.4,0.0,10.8,10.6,46.0,42.0,22.0,1012.3,1009.2,1.0,6.0,0,0.0,0
44895,21.9,0.0,11.4,12.2,31.0,37.0,22.0,1012.7,1009.1,1.0,5.0,0,0.0,0
76709,24.2,0.0,11.2,8.4,35.0,19.0,15.0,1010.7,1007.4,1.0,6.0,0,0.0,0
93239,27.1,0.0,13.0,0.0,43.0,26.0,19.0,1007.7,1007.4,8.0,8.0,0,0.0,0
69116,23.3,0.0,9.8,12.6,41.0,33.0,15.0,1011.3,1009.9,3.0,1.0,0,0.0,0
82344,16.1,0.0,14.6,13.2,37.0,25.0,9.0,1013.3,1009.2,1.0,1.0,0,0.0,0
73442,19.0,0.0,12.0,12.3,48.0,46.0,28.0,1008.3,1004.0,1.0,5.0,0,0.0,0
60154,19.7,0.0,11.0,12.7,41.0,61.0,14.0,1007.9,1005.8,1.0,5.0,0,0.0,0


In [117]:
X = df.iloc[:, :-1]
T =df.iloc[:, -1]
# T = T.reshape(T.shape[0],1)

X,T = partition(X,T)
X_train = np.asarray(X[0])
X_test = np.asarray(X[1])
T_train = list(T[0])
# T_train = T_train.reshape(T_train.shape[0],1)
T_test = list(T[1])
# T_test = T_test.reshape(T_test.shape[0],1)

In [118]:
# T_train
T_train = pd.DataFrame(T_train, columns = ['RainTomorrow0']) 
T_test = pd.DataFrame(T_test, columns = ['RainTomorrow0']) 

In [119]:
T_train['RainTomorrow1'] = 0
T_test['RainTomorrow1'] = 0

In [120]:
# T_train
T_train['RainTomorrow1'] = np.where((T_train['RainTomorrow0'] == 1), 1, 0)
T_train['RainTomorrow0'] = np.where((T_train['RainTomorrow1'] == 0), 1, 0)

T_test['RainTomorrow1'] = np.where((T_test['RainTomorrow0'] == 1), 1, 0)
T_test['RainTomorrow0'] = np.where((T_test['RainTomorrow1'] == 0), 1, 0)

In [121]:
T_train = np.asarray(T_train)
T_test = np.asarray(T_test)

In [122]:
T_test

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [147]:
# nn = NeuralNetLogReg([X_train.shape[1],4,T_train.shape[1]])
# nn.train(X_train,T_train, ftracep=True)
# Ytest, Z = nn.use(X_test, retZ=True)

clsf = NeuralNetLogReg([X_train.shape[1],4,T_train.shape[1]])
clsf.train(X_train,T_train, ftracep=True)

# checking the training error only
classes, Y = clsf.use(X_test, retZ=True)

In Forward t: [[ 0.1172203  -0.00437048]
 [ 0.13201617  0.00480313]
 [ 0.10233019  0.01324763]
 ...
 [ 0.03343507  0.05987151]
 [ 0.04209573  0.05059798]
 [ 0.04403317  0.03781863]]
In Forward t: [[ 0.1172203  -0.00437048]
 [ 0.13201617  0.00480313]
 [ 0.10233019  0.01324763]
 ...
 [ 0.03343507  0.05987151]
 [ 0.04209573  0.05059798]
 [ 0.04403317  0.03781863]]
(4, 45136)
(13, 45136)
In Forward t: [[ 0.1172203  -0.00437048]
 [ 0.13201617  0.00480313]
 [ 0.10233019  0.01324763]
 ...
 [ 0.03343507  0.05987151]
 [ 0.04209573  0.05059798]
 [ 0.04403317  0.03781863]]
(4, 45136)
(13, 45136)
In Forward t: [[-195721.39915618 -267874.43904351]
 [-152391.92072055 -280576.67946008]
 [-195721.39915618 -267874.43904351]
 ...
 [-292680.58491776 -287559.38331062]
 [-195721.39915618 -267874.43904351]
 [-152391.92072055 -280576.67946008]]
In Forward t: [[-1.61345138 -2.78778149]
 [-1.46143934 -2.83330493]
 [-1.75720849 -2.73114367]
 ...
 [-2.84943106 -2.42590099]
 [-1.97730148 -2.68252119]
 [-1.6514140

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174

...

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174

In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[ 0.01235336 -0.14319982]
 [ 0.03411276 -0.13913744]
 [-0.00930966 -0.12174504]
 ...
 [-0.103405   -0.06197843]
 [-0.0822786  -0.08226055]
 [-0.06832283 -0.1058723 ]]
In Forward t: [[-0.08391083 -0.08665115]
 [-0.06446677 -0.10509181]
 [-0.04022137 -0.12417

In [148]:
classes

array([[0.73948341, 0.18924222],
       [0.74760868, 0.18153624],
       [0.75774034, 0.17356088],
       ...,
       [0.76394351, 0.17896103],
       [0.75036528, 0.1929915 ],
       [0.74456947, 0.19180614]])

In [156]:
Y

[array([[-1.4530289 , -0.30503556, -0.79337536, ..., -1.31164582,
         -0.54156729, -0.27200148],
        [-1.46931035, -0.30503556, -0.84746666, ..., -1.69289853,
         -0.54156729, -0.27200148],
        [-1.11111844, -0.30503556, -0.68519277, ..., -1.69289853,
         -0.54156729, -0.27200148],
        ...,
        [ 1.24969185, -0.30503556,  0.07208538, ..., -1.69289853,
         -0.54156729, -0.27200148],
        [ 1.05431445, -0.30503556,  0.23435926, ..., -1.31164582,
         -0.54156729, -0.27200148],
        [ 1.1682846 , -0.30503556,  0.07208538, ...,  0.21336502,
         -0.54156729, -0.27200148]]),
 array([[-0.26847224, -0.01081236,  0.21582086, -0.05164117],
        [-0.14223129, -0.18787586,  0.19017969, -0.1562787 ],
        [-0.00062708, -0.29054608,  0.2462539 , -0.07977892],
        ...,
        [ 0.15721836, -0.0820267 ,  0.20411569,  0.23757227],
        [-0.08518533,  0.10058733,  0.15195224,  0.25853926],
        [-0.01143542,  0.17818161,  0.09456049,  0

In [153]:
Yl = np.argmax(Y, 1)
Tl = np.argmax(T_test, 1)

ValueError: could not broadcast input array from shape (11284,13) into shape (11284)

In [154]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Tl,Yl)

In [152]:
accuracy

0.8005140021269054

In [2]:
t = []


In [3]:
type(t) == list

True

In [1]:
import sklearn.utils
import pandas as pd
import numpy as np
from nn import NeuralNet
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("Hour.csv")
df = df.drop(['dteday', 'instant'], axis = 1)

In [3]:

#, ratio=[0.2,0.2,0.2,0.2,0.2]

def k_folds(df, folds):
    rnum = 1/folds;
    ratio = [rnum] * folds
    assert(np.sum(ratio) == 1)
         # shuffle the data indices 
#     idxs = np.random.permutation(X.index)
    df = sklearn.utils.shuffle(df)
    df = df.reset_index(drop=True)
    X = df.iloc[:, :-1]
    T =df.iloc[:, -1]
        # the number of samples 
    N = X.shape[0]
    
    Xc = []
    Tc = []
    i = 0  # first index to zero
    r = 1/5
    nrows = int(round(N * r))  # number of rows
    
    for k in range(len(ratio)):
        
        Xc.append(X.iloc[i:i+nrows, :])
        Tc.append(T.iloc[i:i+nrows, ])
        i += nrows    
    return Xc, Tc


In [4]:
# Xc, Tc = None
Xc, Tc = k_folds(df, 5)


parameters = [{'Lambda': 0.01, 'niter': 1500, 'nlayers': [3,4]},
             {'Lambda': 0.1, 'niter': 2000, 'nlayers': [2,5]},
             {'Lambda': 1, 'niter': 2500, 'nlayers': [4]}]

Lambda_p = [10, 1, 0.1, 0.001]
nitertns = [1000, 1500, 2000, 3000]
nlay = [[3,4], [2,5], [4]]

def cros_validation(Xc, Tc):
#     Tc = Tc.reshape
    paramarch = []
    testErrs = [] 
    prev_valid = 0
#     train_X = []
#     train_T = []
    
    
    for i in range(len(Xc)):
        res = []
        hype = []
        v_indx = list(range(len(Xc)))
        t_indx = list(range(len(Xc)))
        
        v_indx.remove(i)
        t_indx.remove(i)
        for j in v_indx:
            t_indx.remove(j)
            train_X = None
            train_T = None
#             p = 0
#             print(Xc)

#             print(train_X)
#             print(t_indx)
            for k in t_indx:
                train_X = pd.concat([pd.DataFrame(Xc[k]), train_X], ignore_index=True)
                train_T = pd.concat([pd.DataFrame(Tc[k]), train_T], ignore_index=True)
                
#             print("mtrain_X shape:", train_X.shape)
            train_X =  np.asarray(train_X)
#             print(train_X)
            train_T = np.asarray(train_T)
            train_T = train_T.reshape(train_T.shape[0],1)
            
            for a in Lambda_p:
                for b in nitertns:
                    for c in nlay:
                        hyper = None
                        if(len(c) > 1):
#                             print(train_X.shape)
                            nn = NeuralNet([train_X.shape[1],c[0],c[1],train_T.shape[1]])                           
                            hyper = str(2) + str(c[0]) + str(c[1])
                        else:
                            nn = NeuralNet([train_X.shape[1],c[0],train_T.shape[1]])
                            hyper = str(1) + str(c[0])
                            
#                         print("b:",b)
                        hyper += str(b)    
                        hyper += str(a)
#                         print(train_X)
                        nn.train(train_X,train_T, ftracep=True, Lambda= a, niter = b)
                        Ytest, Z = nn.use(Xc[j], retZ=True)
                        lin_mse = mean_squared_error(Tc[j], Ytest)
                        lin_rmse = np.sqrt(lin_mse)
                        res.append(lin_rmse)
                        hype.append(hyper)
            t_indx.append(j)
                        
              
        print("Res: ", res)
        min_index = res.index(min(res))
        arch = hype[min_index]

        t_indx = list(range(len(Xc)))
#         print("z t_index len", len(t_indx))
        new_train_X= None
        new_train_T = None
        
        for k in t_indx:
#             print(new_train_X)
            new_train_X = pd.concat([pd.DataFrame(Xc[k]), new_train_X], ignore_index=True)
            new_train_T = pd.concat([pd.DataFrame(Tc[k]), new_train_T], ignore_index=True)
        
        new_train_X =  np.asarray(new_train_X)
#             print(new_train_X)
        new_train_T = np.asarray(new_train_T)
        new_train_T = new_train_T.reshape(new_train_T.shape[0],1)
            
        if(arch[0] == '2'):
            l = int(arch[1])
            m = int(arch[2])
            n = int(arch[3:7])
            o = float(arch[7:])
            nn = NeuralNet([new_train_X.shape[1],l,m,new_train_T.shape[1]])
        else:
            l = int(arch[1])
            n = int(arch[2:6])
            o = float(arch[6:])
            nn = NeuralNet([new_train_X.shape[1],l,new_train_T.shape[1]])
            
        nn.train(new_train_X,new_train_T, ftracep=True, Lambda= o, niter = n)
        Ytest, Z = nn.use(Xc[i], retZ=True)
        lin_mse = mean_squared_error(Tc[i], Ytest)
        lin_rmse = np.sqrt(lin_mse)
        testErrs.append(tuple((arch,lin_rmse)))
    return testErrs
                

                

In [5]:
testErrs = cros_validation(Xc, Tc)

Res:  [179.2794533431054, 179.27945305195115, 178.99813820060675, 179.27945426788773, 179.27945335176364, 178.63596331288363, 179.27945341899604, 179.27945338601523, 178.7743833413518, 179.2794533743143, 179.27945337747667, 179.0576156151329, 179.27945338399564, 179.2794533788338, 175.89980017678502, 179.2794533763998, 179.27945337688018, 178.33943769757255, 179.27945337678273, 179.2794533845948, 176.7477530791836, 179.27945337866063, 179.2794533814154, 177.4343593305943, 179.01460470309908, 178.61134861843053, 49.668850124361576, 178.6109886214873, 178.91922991170247, 54.108128663660985, 178.96024267841045, 179.23705238795486, 54.8118988988222, 179.19736781789823, 178.83807604349803, 56.29710115249248, 9.013092470201823, 4.323718772511739, 3.9220639386040697, 3.5051271365844623, 9.452449844584947, 2.6738939647619517, 9.210281852927096, 4.336058489839123, 3.0161489697414634, 4.032211585691162, 4.328150151708349, 3.3652481303633786, 180.72697467073561, 180.7269746934215, 180.82724187583

Res:  [183.6466327727201, 183.64663588753461, 183.47799941364545, 183.64663696159084, 183.6466386647096, 183.1914200845112, 183.64664063174448, 183.64663647281694, 183.4861547216828, 183.64663630452026, 183.64663682436904, 183.4588319960732, 183.64663715628987, 183.64663705632202, 180.63402035185766, 183.64663703489114, 183.646637135959, 179.0240180529756, 183.64663716237476, 183.64663678571773, 179.3337613768911, 183.64663700885257, 183.64663711397873, 181.1571719941615, 183.48338021237686, 183.58371995704312, 51.12143927416161, 182.98643144216052, 183.4379012681081, 57.915355972264656, 183.41890621998797, 183.4800943650102, 51.97869252232927, 183.47521395204015, 183.55889479766805, 53.987322530040366, 4.485899458749933, 11.49172443604901, 3.6863686609168824, 10.129361032937288, 10.180060774391197, 7.455135739798188, 4.82047622676295, 11.332890055482908, 4.369681722896953, 5.029907117995417, 10.72032652229805, 3.842978814481756, 179.28130799611213, 179.28130714363178, 179.479117819381

In [6]:
testErrs

[('1415000.001', 3.6508088532875145),
 ('1410000.001', 3.2849027908084496),
 ('1420000.001', 7.520455775128686),
 ('1420000.001', 7.263427829935096),
 ('1415000.001', 4.367724351933412)]